## Imports

In [212]:
import os
import shutil
import math
import heapq
import numpy as np
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
from collections import defaultdict

## Settings

In [213]:
num_lowest = 60
scores_file_path = './overlap_scores.txt'
labels_dir = './labels'
src_dir = './images'
dst_dir = './lowest'
score_param_names = ['class', 'iou', 'confidence']
vertices = [f'v{i}' for i in range(1, 9)]
score_param_names.extend(vertices)
label_param_names = [v for v in vertices]
label_param_names.extend(['class'])
blacks = defaultdict(int) # 'blacks' are negative scores
gap = defaultdict(int)
clserr = defaultdict(int)
fp = defaultdict(int)
fn = defaultdict(int)
image_scores = defaultdict(list)
image_labels = defaultdict(list)

## Utility function to display an image

In [214]:
def display(img, title):
    plt.imshow(img)
    plt.title(title)
    plt.show()

## Build dict with scores of each image

In [215]:
with open(scores_file_path, 'r') as scores_file:
    for scores in scores_file:
        score_params = {}
        vals = scores.strip().split(' ')
        for i, p in enumerate(vals[1:]):
            score_params[score_param_names[i]] = p
        image_scores[vals[0]].append(score_params)

## Build dict with labels of each image

In [216]:
im_fnames = os.listdir(labels_dir) 
for im_fname in im_fnames:
    imname = im_fname.split('.')[0]
    label_file_name = os.path.join(labels_dir, im_fname)
    with open(label_file_name, 'r') as labels_file:
        for labels in labels_file:
            label_params = {}
            vals = labels.strip().split(' ')
            for i, p in enumerate(vals[:9]):
                label_params[label_param_names[i]] = p
            image_labels[imname].append(label_params)

## Add "black point" according to the difference in no. of identified objects

In [217]:
imnames = list(image_scores.keys())

for imname in imnames:
    score_objects = len(image_scores[imname])
    label_objects = len(image_labels[imname])

## Utility function to find the center point of an obb

In [218]:
def find_center(vertices):
    size = len(vertices)
    vxy = ([[vertices[i], vertices[i+1]] for i in range(0, size, 2)])
    min_x = min([v[0] for v in vxy])
    max_x = max([v[0] for v in vxy])
    min_y = min([v[1] for v in vxy])
    max_y = max([v[1] for v in vxy])

    return [(min_x+max_x)/2, (min_y+max_y)/2]

## Utility function to find the euclidean distance between 2 points

In [219]:
def find_dist(c1, c2):
    return math.sqrt((c1[0]-c2[0])**2. + (c1[1]-c2[1])**2)

## Calculate black-points of every image

In [220]:
imnames = list(image_scores.keys())

for imname in imnames:
    # Assign a black_points for the difference in no. of found objects
    score_objects = len(image_scores[imname])
    label_objects = len(image_labels[imname])
    blacks[imname] += abs(score_objects - label_objects)
    gap[imname] += abs(score_objects - label_objects)
    
    # Find upper common denominator of objects in both score/label of the image
    common = min(score_objects, label_objects)
    
    # Find the nearest labeled object to every scored object
    # Add a black point if the classes of 2 mostly-aligned objects are different
    score_centers = []
    label_centers = []
    for i in range(score_objects):
        score_vertices = [float(image_scores[imname][i][f'v{j}']) for j in range(1, 9)]
        score_centers.append(find_center(score_vertices))
    for i in range(label_objects):
        label_vertices = [int(image_labels[imname][i][f'v{j}']) for j in range(1, 9)]
        label_centers.append(find_center(label_vertices)) 
    dists = []
    for i, score_center in enumerate(score_centers):
        for j, label_center in enumerate(label_centers):
            dist = find_dist(score_center, label_center)
            heapq.heappush(dists, (dist, i, j))
            
    smallest = heapq.nsmallest(common, dists)
    for s in smallest:
        i = s[1]
        j = s[2]
        if image_scores[imname][i]['class'] != image_labels[imname][j]['class']:
            blacks[imname] += 1
            clserr[imname] += 1
            
    # Add a black point in case confidence < 0.7 and iou > 0.5
    for i in range(score_objects):
        if float(image_scores[imname][i]['confidence']) < 0.7 and float(image_scores[imname][i]['iou']) > 0.5:
            blacks[imname] += 1
            fp[imname] += 1
            
    # Add a black point in case confidence > 0.3 and iou < 0.5
    for i in range(score_objects):
        if float(image_scores[imname][i]['confidence']) > 0.3 and float(image_scores[imname][i]['iou']) < 0.5:
            blacks[imname] += 1
            fn[imname] += 1

## Make a list of lowest-performing (= highest black-points scoring) images

In [221]:
sorted_by_score = list(blacks.items())
sorted_by_score.sort(key=lambda x:x[1], reverse=True)
lowest = sorted_by_score[:num_lowest]

## Draw label & score bboxes/classes on image and store it

In [222]:
debug_image = 'v4_frame_000669'
imnames = [l[0] for l in lowest]

for imname in tqdm(imnames):
    label_objects = len(image_labels[imname])
    score_objects = len(image_scores[imname])
    fname = f'{imname}.png'
    img = cv2.imread(os.path.join(src_dir, fname))

    for i in range(label_objects):
        x1 = int(image_labels[imname][i]['v1'])
        y1 = int(image_labels[imname][i]['v2'])
        x2 = int(image_labels[imname][i]['v3'])
        y2 = int(image_labels[imname][i]['v4'])
        x3 = int(image_labels[imname][i]['v5'])
        y3 = int(image_labels[imname][i]['v6'])
        x4 = int(image_labels[imname][i]['v7'])
        y4 = int(image_labels[imname][i]['v8'])

        pts = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        pts = np.array(pts, dtype=np.int32)
        _ = cv2.polylines(img, [pts], True, (0, 255, 0), 2)
        object_name = image_labels[imname][i]['class']
        x = int((x1 + x2 + x3 + x4) / 4)
        y = int((y1 + y2 + y3 + y4) / 4) + 30
        _ = cv2.putText(img, object_name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    for i in range(score_objects):
        if imname == debug_image:
            print(f'score_objects: {score_objects}')
        x1 = float(image_scores[imname][i]['v1'])
        y1 = float(image_scores[imname][i]['v2'])
        x2 = float(image_scores[imname][i]['v3'])
        y2 = float(image_scores[imname][i]['v4'])
        x3 = float(image_scores[imname][i]['v5'])
        y3 = float(image_scores[imname][i]['v6'])
        x4 = float(image_scores[imname][i]['v7'])
        y4 = float(image_scores[imname][i]['v8'])

        pts = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        pts = np.array(pts, dtype=np.int32)
        _ = cv2.polylines(img, [pts], True, (0, 0, 255), 2)
        confidence = float(image_scores[imname][i]["confidence"])
        object_info = f'{image_scores[imname][i]["class"]} {confidence:.1f}'
        if imname == debug_image:
            print(f'i: {i}, object_info: {object_info}')
        x = int((x1 + x2 + x3 + x4) / 4)
        y = int((y1 + y2 + y3 + y4) / 4)
        _ = cv2.putText(img, object_info, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
    dst_name = f'{fname}_{blacks[imname]}_gap_{gap[imname]}_clserr_{clserr[imname]}_fp_{fp[imname]}_fn_{fn[imname]}.png'
    dst = os.path.join(dst_dir, dst_name)
    cv2.imwrite(dst, img)

  5%|▌         | 3/60 [00:00<00:02, 22.95it/s]

score_objects: 6
i: 0, object_info: person 1.0
score_objects: 6
i: 1, object_info: person 0.9
score_objects: 6
i: 2, object_info: pallet 1.0
score_objects: 6
i: 3, object_info: forklift 1.0
score_objects: 6
i: 4, object_info: forklift 1.0
score_objects: 6
i: 5, object_info: forklift 0.8


100%|██████████| 60/60 [00:02<00:00, 25.69it/s]


In [226]:
image_scores['v4_frame_000669'][1]['confidence']

'0.8575'

In [230]:
image_labels['v4_frame_000669'][3]['v8']

'386'